In [1]:
import pandas as pd
import re
import nltk
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt_tab')
nltk.download('punkt')

C:\Program Files\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Nukuta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nukuta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Параметры чанков
MAX_SENTENCES_PER_CHUNK = 5
OVERLAP = 1

In [3]:
df = pd.read_csv("papers.csv")

In [4]:
def clean_text(text):
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [5]:
df["clean_text"] = df["Текст статьи"].apply(clean_text)

Разбиваю на чанки по предложениям с перекрытием

In [6]:
def chunk_text_by_sentences(text, max_sentences=5, overlap=1):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    start = 0
    while start < len(sentences):
        end = start + max_sentences
        chunk_sents = sentences[start:end]
        chunk_text = " ".join(chunk_sents)
        chunks.append(chunk_text)
        new_start = end - overlap
        if new_start <= start:
            break
        start = new_start
    return chunks

In [7]:
docs = []
for idx, row in df.iterrows():
    title = row["Заголовок"]
    text = row["clean_text"]
    text_chunks = chunk_text_by_sentences(text, max_sentences=MAX_SENTENCES_PER_CHUNK, overlap=OVERLAP)
    for ch in text_chunks:
        docs.append({
            "title": title,
            "text_chunk": ch
        })

In [9]:
texts_for_embedding = [(d["title"], d["text_chunk"]) for d in docs]

In [11]:
texts_for_embedding[0]

('Правильный способ обработки данных смешанного типа.Современные метрики расстояния.',
 'Забавный факт: Scikit-learn не имеет никаких метрик расстояния, которые могут обрабатывать как категориальные, так и непрерывные данные!Как мы можем затем использовать алгоритмы кластеризации, например,K-NN, если у нас есть набор данных с переменными смешанного типа? Фото Феди Джейкоб на Unsplash Обновление (27/07/19) - пакет был выпущен в PYPI в качестве Distython.Я опубликовал статью, чтобы объяснить, как она работает. Большая проблема, с которой я столкнулся во время летней стажировки в ИТ-инновационном центре, была отсутствие существующих реализаций метрик дистанции, которые могли бы обрабатывать как данные смешанного типа, так и отсутствующие значения.Он начал мой долгий поиск алгоритмов, которые могут удовлетворить эти требования.Несколько исследовательских работ позже я обнаружил довольно интересные показатели дистанции, которые могут помочь повысить точность вашей модели машинного обучения 

In [14]:
df_save = pd.DataFrame(texts_for_embedding, columns=["Title", "Text Chunk"])
df.to_csv("to_process.csv", index=False, encoding='utf-8')

Далее я собираюсь пойти через генерацию вопросов к каждому чанку с помощью LLM. Из-за сжатых сроков воспользовался API chatgpt, чтобы получить к каждому чанку по 5 заголовков
Скрипт приложен в архиве generate_questions.py

Импортирую полученный файл

In [15]:
df = pd.read_csv("partial_results.csv")

In [16]:
def clean_question(q):
    return q.strip()

In [17]:
df["question_clean"] = df["question"].apply(clean_question)

Добавляю к новым вопросам текущие заголовки, чтобы в дальнейшем у одного чанка было 6 признаков

In [34]:
df['question_clean'] = df['question'] 

new_rows = df[['title', 'chunk']].drop_duplicates().copy()
new_rows['question'] = new_rows['title']  
new_rows['question_clean'] = new_rows['title'] 
final_df = pd.concat([df, new_rows], ignore_index=True)

Теперь есть датасет, разбитый на чанки, у каждого чанка есть 6 строк со списками вопросов к этому чанку.

In [37]:
questions = final_df["question_clean"].tolist()

In [41]:
cleaned_questions = [
    q.split('. ', 1)[1] if '. ' in q else q for q in questions
]

In [43]:
model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = model.encode(questions, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

Batches:   0%|          | 0/2095 [00:00<?, ?it/s]

In [45]:
np.save('embeddings_titles.npy', embeddings)

### Индекс для эмбеддингов

In [46]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

In [47]:
def retrieve_top_k_embeddings(query, top_k=5):
    q_vec = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    distances, indices = index.search(q_vec, top_k)
    return [(int(idx_), float(distances[0][i])) for i, idx_ in enumerate(indices[0])]

In [48]:
def compute_embedding_score(query, doc_idx):
    q_vec = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    doc_vec = embeddings[doc_idx].reshape(1, -1)
    score = float((q_vec @ doc_vec.T)[0][0])
    return score

### Матрица TF-IDF

In [49]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(questions)

In [50]:
from sklearn.metrics.pairwise import cosine_similarity
def retrieve_top_k_tfidf(query, top_k=5):
    query_vec = tfidf_vectorizer.transform([query])
    sims = cosine_similarity(query_vec, tfidf_matrix)[0]
    top_indices = np.argpartition(sims, -top_k)[-top_k:]
    top_indices = top_indices[np.argsort(-sims[top_indices])]
    return [(int(idx), float(sims[idx])) for idx in top_indices]

In [51]:
def compute_tfidf_score(query, doc_idx):
    query_vec = tfidf_vectorizer.transform([query])
    doc_vec = tfidf_matrix[doc_idx]
    sims = cosine_similarity(query_vec, doc_vec)[0][0]
    return float(sims)

### BM25

In [52]:
from rank_bm25 import BM25Okapi
tokenized_questions = [q.split() for q in questions]
bm25 = BM25Okapi(tokenized_questions)

In [53]:
def retrieve_top_k_bm25(query, top_k=5):
    tokenized_query = query.split()
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argpartition(scores, -top_k)[-top_k:]
    top_indices = top_indices[np.argsort(-scores[top_indices])]
    return [(int(idx), float(scores[idx])) for idx in top_indices]

In [54]:
def compute_bm25_score(query, doc_idx):
    tokenized_query = query.split()
    scores = bm25.get_scores(tokenized_query)
    return float(scores[doc_idx])

In [165]:
def retrieve_ensemble(query, top_k=5):
    # Получаем топ-5 от каждого метода
    emb_results = retrieve_top_k_embeddings(query, top_k=top_k)
    tfidf_results = retrieve_top_k_tfidf(query, top_k=top_k)
    bm25_results = retrieve_top_k_bm25(query, top_k=top_k)

    # Собираем все индексы
    candidate_indices = set([x[0] for x in emb_results] + [x[0] for x in tfidf_results] + [x[0] for x in bm25_results])

    # Для каждого кандидата считаем все три скоринга
    final_results = []
    for idx_ in candidate_indices:
        emb_score = compute_embedding_score(query, idx_)
        tfidf_score = compute_tfidf_score(query, idx_)
        bm25_score = compute_bm25_score(query, idx_)

        tfidf_weight = 1.0
        emb_weight = 1.0
        final_score = (emb_score * emb_weight + tfidf_score * tfidf_weight + bm25_score) / (emb_weight + tfidf_weight + 1)
        final_results.append((idx_, final_score, emb_score, tfidf_score, bm25_score))

    final_results.sort(key=lambda x: x[1], reverse=True)
    return final_results

In [162]:
def get_all_chunks_for_user_question(user_query, top_k=5):
    ensemble_results = retrieve_ensemble(user_query, top_k=top_k)
    candidates = []

    for r in ensemble_results:
        doc_idx, final_score, emb_s, tfidf_s, bm25_s = r

        title = final_df.iloc[doc_idx]["title"]
        chunk = final_df.iloc[doc_idx]["chunk"]
        question = final_df.iloc[doc_idx]["question"]

        candidates.append({
            "doc_idx": doc_idx,
            "final_score": final_score,
            "emb_score": emb_s,
            "tfidf_score": tfidf_s,
            "bm25_score": bm25_s,
            "title": title,
            "chunk": chunk,
            "question": question
        })
    sorted_candidates = sorted(candidates, key=lambda x: x["final_score"], reverse=True)

    return sorted_candidates[:top_k]

In [153]:
user_query = "Что такое обучение с подкреплением?"
top_k = 3

all_candidates = get_all_chunks_for_user_question(user_query, top_k=top_k)

print("\nВсе кандидаты после ансамблирования:")
for candidate in all_candidates:
    print(f"doc_idx={candidate['doc_idx']}, final_score={candidate['final_score']:.4f}, "
          f"emb = {candidate['emb_score']:.4f}, tfidf={candidate['tfidf_score']:.4f}, bm25={candidate['bm25_score']:.4f}")
    print(f"Похожий вопрос: {candidate['question']}")
    print(f"Статья: {candidate['title']}")
    print(f"Соответствующий чанк: {candidate['chunk']}")
    print("-" * 80)


Все кандидаты после ансамблирования:
doc_idx=55786, final_score=0.9973, emb = 0.7316, tfidf=0.6003, bm25=17.5623
Похожий вопрос: 2. Что такое возврат в контексте обучения с подкреплением?
Статья: Подкрепление обучения: процесс назначения Марков (часть 1)
Соответствующий чанк: Награда и возвращение Награды - это численные значения, которые агент получает при выполнении какого -либо действия в некоторых штатах в окружающей среде.Числовое значение может быть положительным или отрицательным в зависимости от действий агента.В обучении подкрепления мы заботимся о максимизации совокупного вознаграждения (все агент по вознаграждению получает из окружающей среды) Вместо того, чтобы агент вознаграждения получает от текущего состояния (также называемого немедленным вознаграждением).Эта общая сумма вознаграждения, которую агент получает из окружающей среды, называется возвратами. Мы можем определить возврат как: Возврат (полное вознаграждение от окружающей среды) R [T+1] - это вознаграждение, пол

Evaluation

In [154]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

qa_tokenizer = AutoTokenizer.from_pretrained("KirrAno93/rubert-base-cased-finetuned-squad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("KirrAno93/rubert-base-cased-finetuned-squad")
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)

Device set to use cpu


In [155]:
def generate_answer(question, top_docs):
    context = " ".join([doc["chunk"] for doc in top_docs])
    result = qa_pipeline(question=question, context=context)
    return result["answer"]

In [166]:
question = ["Для чего можно использовать сверточные нейронные сети?",
            "Что такое обучение с подкреплением?",
            "Как развернуть модель машинного обучения?",
            "Как написать алгоритм случайного леса?"]
top_k = 4
for q in question:
    all_candidates = get_all_chunks_for_user_question(q, top_k=top_k)
    #print(all_candidates)
    answer = generate_answer(q, all_candidates)
    print(f"Вопрос: {q} \nОтвет на вопрос: {answer} \n\n")


Вопрос: Для чего можно использовать сверточные нейронные сети? 
Ответ на вопрос: для классификации образцов на несколько уникальных классов 


Вопрос: Что такое обучение с подкреплением? 
Ответ на вопрос: агент, который взаимодействует с окружающей средой 


Вопрос: Как развернуть модель машинного обучения? 
Ответ на вопрос: в конечной точке 


Вопрос: Как написать алгоритм случайного леса? 
Ответ на вопрос: вы можете предоставить диапазон для количества деревьев между 10 и 50 




Для улучшения сделал бы еще ансамбль поиска не только в заголовках, но и в текстах. Пробовал просто поиск по текстовым чанкам, получалось хуже, но если по заголовкам находить целый текст, вероятно, будет лучше

In [143]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# model_path = "PleiAs"
 # qa_tokenizer = AutoTokenizer.from_pretrained(model_path)
# qa_model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

In [144]:
# def generate_answer_from_summaries(context, query, tokenizer, model):
#     prompt = f"Ответь на вопрос, опираясь на контекст, если в контексте нет ответа на вопрос, отвечай не знаю :\n\nContext: {context}\n\nВопрос: {query}\nОтвет:"
#     inputs = tokenizer(prompt, return_tensors="pt")
#     inputs.pop("token_type_ids", None) 
#     output = model.generate(**inputs, max_length=1000, num_return_sequences=1)
# 
#     return tokenizer.decode(output[0], skip_special_tokens=True)

In [145]:
# question = "Как написать алгоритм случайного леса?"
# all_candidates = get_all_chunks_for_user_question(question, top_k=top_k)
# context = "\n\n".join([doc["chunk"] for doc in all_candidates])
# 
# response = generate_answer_from_summaries(context, question, qa_tokenizer, qa_model)
# print(f"\nОтвет на вопрос: {response}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Ответ на вопрос: Ответь на вопрос, опираясь на контекст, если в контексте нет ответа на вопрос, отвечай не знаю :

Context: Этот код создает случайный поиск для определения лучших параметров для случайного леса для выполнения своих классификаций. В следующем коде используется приведенный выше обычный поиск в сетке, запускает 100 различных комбинаций моделей и идентифицирует лучший. Это лучшие параметры: {‘N_estimators’: 266, ‘min_samples_split’: 5, ‘min_samples_leaf’: 1, «max_features’: «sqrt», «max_depth»: 30, «bootstrap»: true} Используя лучшую возможную модель случайного леса, мы достигаем точности 68,97%. Этот балл находится на одном уровне с результатами логистической регрессии и работает хуже, чем KNN. На мой взгляд, наиболее полезный результат случайного леса - это важность особенности.

Вы можете запустить каждый из экспериментов параллельно. Минусы: вычислительно дорогие, так как строится так много моделей. Если конкретный гиперпараметр не важен, вы излишне исследуете различн